In [ ]:
import numpy as np
import pandas as pd
np.random.seed(seed=12345)

在这篇附录中，我们会深入探究NumPy库的数组计算。这将会包含更多ndarray的细节和更高级的数组操作和算法。

## ndarray对象的内部机理
NumPy的ndarray提供了一种将同质数据块（可以是连续或跨越）解释为多维数组对象的方式。正如你之前所看到的那样，数据类型（dtype）决定了数据的解释方式，比如浮点数、整数、布尔值等。

ndarray如此强大的部分原因是所有数组对象都是数据块的一个跨度视图（strided view）。你可能想知道数组视图arr[::2,::-1]不复制任何数据的原因是什么。简单地说，ndarray不只是一块内存和一个dtype，它还有跨度信息，这使得数组能以各种步幅（step size）在内存中移动。更准确地讲，ndarray内部由以下内容组成：

* 一个指向数据（内存或内存映射文件中的一块数据）的指针。
* 数据类型或dtype，描述在数组中的固定大小值的格子。
* 一个表示数组形状（shape）的元组。
* 一个跨度元组（stride），其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数。
图A-1简单地说明了ndarray的内部结构。
![ndarray](./figures/ndarray.webp)

例如，一个10×5的数组，其形状为(10,5)：

In [ ]:
np.ones((10, 5)).shape

一个典型的（C顺序，稍后将详细讲解）3×4×5的float64（8个字节）数组，其跨度为(160,40,8) —— 知道跨度是非常有用的，通常，跨度在一个轴上越大，沿这个轴进行计算的开销就越大：

In [ ]:
np.ones((3, 4, 5), dtype=np.float64).strides

虽然NumPy用户很少会对数组的跨度信息感兴趣，但它们却是构建非复制式数组视图的重要因素。跨度甚至可以是负数，这样会使数组在内存中后向移动，比如在切片obj[::-1]或obj[:,::-1]中就是这样的。

## 高级数组操作
除花式索引、切片、布尔条件取子集等操作之外，数组的操作方式还有很多。虽然pandas中的高级函数可以处理数据分析工作中的许多重型任务，但有时你还是需要编写一些在现有库中找不到的数据算法。

### 数组重塑
多数情况下，你可以无需复制任何数据，就将数组从一个形状转换为另一个形状。只需向数组的实例方法reshape传入一个表示新形状的元组即可实现该目的。例如，假设有一个一维数组，我们希望将其重新排列为一个矩阵：

In [ ]:
arr = np.arange(12)
arr

In [ ]:
arr.reshape((4, 3))

In [ ]:
arr.reshape((4,3), order="F")

![numpy_reshape](./figures/numpy_reshape.webp)

多维数组也能被重塑：

In [ ]:
arr.reshape((4, 3)).reshape((3, 4))

作为参数的形状的其中一维可以是－1，它表示该维度的大小由数据本身推断而来：

In [ ]:
arr = np.arange(15)
arr.reshape((5, -1))

与reshape将一维数组转换为多维数组的运算过程相反的运算通常称为扁平化（flattening）或散开（raveling）：

In [ ]:
arr = np.arange(15).reshape((5, 3))
arr

In [ ]:
arr.ravel()

如果结果中的值与原始数组相同，ravel不会产生源数据的副本。flatten方法的行为类似于ravel，只不过它总是返回数据的副本：

In [ ]:
arr.flatten()

像reshape和reval这样的函数，都可以接受一个表示数组数据存放顺序的order参数。一般可以是'C'或'F'（还有'A'和'K'等不常用的选项，具体请参考NumPy的文档）

In [ ]:
arr.ravel('F')

### 数组的合并和拆分
numpy.concatenate可以按指定轴将一个由数组组成的序列（如元组、列表等）连接到一起：

In [ ]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr1

In [ ]:
arr2 = np.array([[7, 8, 9], [10, 11, 12]])
arr2

In [ ]:
np.concatenate([arr1, arr2], axis=0)

In [ ]:
np.concatenate([arr1, arr2], axis=1)

对于常见的连接操作，NumPy提供了一些比较方便的方法（如vstack和hstack）。因此，上面的运算还可以表达为：

In [ ]:
np.vstack((arr1, arr2))

In [ ]:
np.hstack((arr1, arr2))

与此相反，split用于将一个数组沿指定轴拆分为多个数组：

In [ ]:
arr = np.random.randn(5, 2)
arr

In [ ]:
first, second, third = np.split(arr, [1, 3])
first

In [ ]:
second

In [ ]:
third

传入到np.split的值[1,3]指示在哪个索引处分割数组。

下表列出了所有关于数组连接和拆分的函数，其中有些是专门为了方便常见的连接运算而提供的。
![numpy_concat](./figures/numpy_concat.webp)

### 元素的重复操作：tile和repeat
对数组进行重复以产生更大数组的工具主要是repeat和tile这两个函数。repeat会将数组中的各个元素重复一定次数，从而产生一个更大的数组：

In [ ]:
arr = np.arange(3)
arr

In [ ]:
arr.repeat(3)

笔记：跟其他流行的数组编程语言（如MATLAB）不同，NumPy中很少需要对数组进行重复（replicate）。这主要是因为广播（broadcasting，我们将在下一节中讲解该技术）能更好地满足该需求。

默认情况下，如果传入的是一个整数，则各元素就都会重复那么多次。如果传入的是一组整数，则各元素就可以重复不同的次数：

In [ ]:
arr.repeat([2, 3, 4])

对于多维数组，还可以让它们的元素沿指定轴重复：

In [ ]:
arr = np.random.randn(2, 2)
arr

In [ ]:
arr.repeat(2, axis=0)

注意，如果没有设置轴向，则数组会被扁平化，这可能不会是你想要的结果。

In [ ]:
arr.repeat(2)

同样，在对多维进行重复时，也可以传入一组整数，这样就会使各切片重复不同的次数：

In [ ]:
arr.repeat([2, 3], axis=0)

In [ ]:
arr.repeat([2, 3], axis=1)

tile的功能是沿指定轴向堆叠数组的副本。你可以形象地将其想象成“铺瓷砖”：

In [ ]:
arr

In [ ]:
np.tile(arr, 2)

第二个参数是瓷砖的数量。对于标量，瓷砖是水平铺设的，而不是垂直铺设。它可以是一个表示“铺设”布局的元组：

In [ ]:
np.tile(arr, (2, 1))

In [ ]:
np.tile(arr, (3, 2))

### 花式索引的等价函数：take和put
在前面中我们讲过，获取和设置数组子集的一个办法是通过整数数组使用花式索引：

In [ ]:
arr = np.arange(10) * 100
arr

In [ ]:
inds = [7, 1, 2, 6]
arr[inds]

ndarray还有其它方法用于获取单个轴向上的选区：

In [ ]:
arr.take(inds)

In [ ]:
arr.put(inds, 42)
arr

In [ ]:
arr.put(inds, [40, 41, 42, 43])
arr

要在其它轴上使用take，只需传入axis关键字即可：

In [ ]:
inds = [2, 0, 2, 1]
arr = np.random.randn(2, 4)
arr

In [ ]:
arr.take(inds, axis=1)

put不接受axis参数，它只会在数组的扁平化版本（一维，C顺序）上进行索引。因此，在需要用其他轴向的索引设置元素时，最好还是使用花式索引。

### 广播
广播（broadcasting）指的是不同形状的数组之间的算术运算的执行方式。它是一种非常强大的功能，但也容易令人误解，即使是经验丰富的老手也是如此。将标量值跟数组合并时就会发生最简单的广播：

In [ ]:
arr = np.arange(5)
arr

In [ ]:
arr * 4

这里我们说：在这个乘法运算中，标量值4被广播到了其他所有的元素上。

看一个例子，我们可以通过减去列平均值的方式对数组的每一列进行距平化处理。这个问题解决起来非常简单：

In [ ]:
arr = np.random.randn(4, 3)
arr.mean(0)

In [ ]:
arr.shape

In [ ]:
arr.mean(0).shape

In [ ]:
demeaned = arr - arr.mean(0)
demeaned

In [ ]:
demeaned.mean(0)

![numpy_broadcasting](./figures/numpy_broadcasting.webp)

根据该原则，要在1轴向上做减法（即各行减去行平均值），较小的那个数组的形状必须是(4,1)：

In [ ]:
arr

In [ ]:
arr.shape

In [ ]:
row_means = arr.mean(axis=1)
row_means.shape

In [ ]:
arr - row_means

人们经常需要通过算术运算过程将较低维度的数组在除0轴以外的其他轴向上广播。根据广播的原则，较小数组的“广播维”必须为1。在上面那个行距平化的例子中，这就意味着要将行平均值的形状变成(4,1)而不是(4,)：

In [ ]:
row_means.reshape((4, 1)).shape

In [ ]:
demeaned = arr - row_means.reshape((4, 1))
demeaned.mean(1)

对于三维的情况，在三维中的任何一维上广播其实也就是将数据重塑为兼容的形状而已。下图说明了要在三维数组各维度上广播的形状需求。
![broadcasting_axis](./figures/broadcasting_axis.webp)

于是就有了一个非常普遍的问题（尤其是在通用算法中），即专门为了广播而添加一个长度为1的新轴。虽然reshape是一个办法，但插入轴需要构造一个表示新形状的元组。这是一个很郁闷的过程。因此，NumPy数组提供了一种通过索引机制插入轴的特殊语法。下面这段代码通过特殊的np.newaxis属性以及“全”切片来插入新轴：

In [ ]:
arr = np.zeros((4, 4))
arr.shape

In [ ]:
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape

In [ ]:
arr_1d = np.random.normal(size=3)
arr_1d.shape

In [ ]:
arr_1d[:, np.newaxis].shape

In [ ]:
arr_1d[np.newaxis, :].shape

因此，如果我们有一个三维数组，并希望对轴2进行距平化，那么只需要编写下面这样的代码就可以了：

In [ ]:
arr = np.random.randn(3, 4, 5)
arr.shape

In [ ]:
depth_means = arr.mean(2)
depth_means

In [ ]:
depth_means.shape

In [ ]:
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned.mean(2)

### 更多有关排序的话题
跟Python内置的列表一样，ndarray的sort实例方法也是就地排序。也就是说，数组内容的重新排列是不会产生新数组的：

In [ ]:
arr = np.random.randn(6)
arr.sort()
arr

相反，numpy.sort会为原数组创建一个已排序副本。另外，它所接受的参数（比如kind）跟ndarray.sort一样：

In [ ]:
arr = np.random.randn(5)
arr

In [ ]:
np.sort(arr)

In [ ]:
arr

这两个排序方法都可以接受一个axis参数，以便沿指定轴向对各块数据进行单独排序：

In [ ]:
arr = np.random.randn(3, 5)
arr

In [ ]:
arr.sort(axis=1)
arr

你可能注意到了，这两个排序方法都不可以被设置为降序。其实这也无所谓，因为数组切片会产生视图（也就是说，不会产生副本，也不需要任何其他的计算工作）。许多Python用户都很熟悉一个有关列表的小技巧：values[::-1]可以返回一个反序的列表。对ndarray也是如此：

In [ ]:
arr[:, ::-1]

### 间接排序：argsort和lexsort
在数据分析工作中，常常需要根据一个或多个键对数据集进行排序。例如，一个有关学生信息的数据表可能需要以姓和名进行排序（先姓后名）。这就是间接排序的一个例子，如果你阅读过有关pandas的章节，那就已经见过不少高级例子了。给定一个或多个键，你就可以得到一个由整数组成的索引数组（我亲切地称之为索引器），其中的索引值说明了数据在新顺序下的位置。argsort和numpy.lexsort就是实现该功能的两个主要方法。下面是一个简单的例子：

In [ ]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort()
indexer

In [ ]:
values[indexer]

一个更复杂的例子，下面这段代码根据数组的第一行对其进行排序：

In [ ]:
arr = np.random.randn(3, 5)
arr[0] = values
arr

In [ ]:
arr[:, arr[0].argsort()]

lexsort跟argsort差不多，只不过它可以一次性对多个键数组执行间接排序（字典序）。假设我们想对一些以姓和名标识的数据进行排序：

In [ ]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name))
sorter

In [ ]:
list(zip(last_name[sorter], first_name[sorter]))

刚开始使用lexsort的时候可能会比较容易头晕，这是因为键的应用顺序是从最后一个传入的算起的。不难看出，last_name是先于first_name被应用的。

笔记：Series和DataFrame的sort_index以及Series的order方法就是通过这些函数的变体（它们还必须考虑缺失值）实现的。

### 部分排序数组
排序的目的之一可能是确定数组中最大或最小的元素。NumPy有两个优化方法，numpy.partition和np.argpartition，可以在第k个最小元素划分的数组：

In [ ]:
np.random.seed(12345)
arr = np.random.randn(20)
arr

In [ ]:
np.partition(arr, 3)

当你调用partition(arr, 3)，结果中的头三个元素是最小的三个，没有特定的顺序。numpy.argpartition与numpy.argsort相似，会返回索引，重排数据为等价的顺序：

In [ ]:
indices = np.argpartition(arr, 3)
indices

In [ ]:
arr.take(indices)

## The end! :)